# Atividade: Perceptron para classificação de petroleo
**INSTITUTO FEDERAL DE MINAS GERIAS**
*Departamento de Engenharia e Computação*

**Professor:** Ciniro Nametala

**Aluno:** Euler Gomes


In [103]:
#importando bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import pygwalker as pyg
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import plotly.graph_objects as go


### Funções de ativação

In [104]:
#funcoes de ativacao
#funcao degrau bipolar
def degrau_bipolar(x):
    if x > 0:
        y = 1
    elif x < 0:
        y = -1
    return y

### Implementação do Perceptron

In [105]:
#leitura do dataset de treino
data = pd.read_csv('data/dataset_oil_training.csv', sep=';')

data.head()

,Amostra,x1,x2,x3,d
0,1,-0.6508,0.1097,4.0009,-1.0
1,2,-1.4492,0.8896,4.4005,-1.0
2,3,2.0850,0.6876,7.9845,-1.0
3,4,0.2626,1.1476,12.0710,-1.0
4,5,0.6418,1.0234,7.7985,1.0


In [106]:
#quatidade de elementos na amostra
n_amostras = data.shape[0]

#quantidade de variaveis de entrada (subtrair a coluna de tipos)
n_variaveis = data.shape[1] -1

#separando os dados contendo apenas as variaveis de entrada
x = data.iloc[:, 1:n_variaveis].values
print(x)

[[-0.6508  0.1097  4.0009]
 [-1.4492  0.8896  4.4005]
 [ 2.085   0.6876  7.9845]
 [ 0.2626  1.1476 12.071 ]
 [ 0.6418  1.0234  7.7985]
 [ 0.2569  0.673   8.3265]
 [ 1.1155  0.6043  7.4446]
 [ 0.0914  0.3399  7.0677]
 [ 0.0121  0.5256  4.6316]
 [-0.0429  0.466   5.4323]
 [ 0.434   0.687   8.2287]
 [ 0.2735  1.0287  7.1934]
 [ 0.4839  0.4851  7.485 ]
 [ 0.4089 -0.1267  5.5019]
 [ 1.4391  0.1614  8.5843]
 [-0.9115 -0.1973  2.1962]
 [ 0.3654  1.0475  7.4858]
 [ 0.2144  0.7515  7.1699]
 [ 0.2013  1.0014  6.5489]
 [ 0.6483  0.2183  5.8991]
 [-0.1147  0.2242  7.2435]
 [-0.797   0.8795  3.8762]
 [-1.0625  0.6366  2.4707]
 [ 0.5307  0.1285  5.6883]
 [-1.22    0.7777  1.7252]
 [ 0.3957  0.1076  5.6623]
 [-0.1013  0.5989  7.1812]
 [ 2.4482  0.9455 11.2095]
 [ 2.0149  0.6192 10.9263]
 [ 0.2012  0.2611  5.4631]]


In [107]:
#inserindo o bias
bias = np.ones((n_amostras, 1)) * -1

#concatenar o bias com o dataset x
x = np.hstack((x, bias))
print(x)

[[-0.6508  0.1097  4.0009 -1.    ]
 [-1.4492  0.8896  4.4005 -1.    ]
 [ 2.085   0.6876  7.9845 -1.    ]
 [ 0.2626  1.1476 12.071  -1.    ]
 [ 0.6418  1.0234  7.7985 -1.    ]
 [ 0.2569  0.673   8.3265 -1.    ]
 [ 1.1155  0.6043  7.4446 -1.    ]
 [ 0.0914  0.3399  7.0677 -1.    ]
 [ 0.0121  0.5256  4.6316 -1.    ]
 [-0.0429  0.466   5.4323 -1.    ]
 [ 0.434   0.687   8.2287 -1.    ]
 [ 0.2735  1.0287  7.1934 -1.    ]
 [ 0.4839  0.4851  7.485  -1.    ]
 [ 0.4089 -0.1267  5.5019 -1.    ]
 [ 1.4391  0.1614  8.5843 -1.    ]
 [-0.9115 -0.1973  2.1962 -1.    ]
 [ 0.3654  1.0475  7.4858 -1.    ]
 [ 0.2144  0.7515  7.1699 -1.    ]
 [ 0.2013  1.0014  6.5489 -1.    ]
 [ 0.6483  0.2183  5.8991 -1.    ]
 [-0.1147  0.2242  7.2435 -1.    ]
 [-0.797   0.8795  3.8762 -1.    ]
 [-1.0625  0.6366  2.4707 -1.    ]
 [ 0.5307  0.1285  5.6883 -1.    ]
 [-1.22    0.7777  1.7252 -1.    ]
 [ 0.3957  0.1076  5.6623 -1.    ]
 [-0.1013  0.5989  7.1812 -1.    ]
 [ 2.4482  0.9455 11.2095 -1.    ]
 [ 2.0149  0.6192 10

In [108]:
#associar cada entrada xi com cada saída y
y = data.iloc[:, n_variaveis].values
print(y)

#contar quantas amostras de cada classe existem no dataset
print(f"-1: {(y == -1).sum()}")
print(f"1: {(y == 1).sum()}")

[-1. -1. -1. -1.  1.  1. -1.  1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1.
  1.  1. -1.  1.  1.  1.  1. -1. -1.  1. -1.  1.]
-1: 16
1: 14


## Inicialização do perceptron

In [109]:
#inicializar o vetor de pesos W com valores aleatorios
w = np.random.uniform(-1,1, n_variaveis)
print(w)

[-0.42993895  0.2244435   0.31163973 -0.98224845]


In [110]:
#taxa de aprendizagem
eta = 0.01

#tolerancia
tolerancia = 0.01

#erro medio
erro_medio = tolerancia + 1

#vetor de erro medio por epoca
erros_epocas = []

#contador de epocas
epoca = 0

#maximo de epocas para treino
max_epoca = 500

In [111]:
#criterios de parada
criterio_erro = True
criterio_epocas = True

while criterio_erro == True and criterio_epocas == True:
    erro_atual = 0
    epoca = epoca + 1

    #embaralhar os indices
    ind_embaralhados = np.random.permutation(n_amostras)

    for i in range(n_amostras):
        indice_amostra = ind_embaralhados[i]
        xi_amostra = x[indice_amostra, :]

        #potencial e ativacao
        u = np.dot(w.T, xi_amostra)

        yhat = degrau_bipolar(u)

        e = y[indice_amostra] - yhat

        w = w + (eta * e) * xi_amostra

        erro_atual = erro_atual + np.abs(e)

    erro_medio = erro_atual / n_amostras
    erros_epocas.append(erro_medio)

    if erro_medio < tolerancia:
            criterio_erro = False
    if epoca > max_epoca:
            criterio_epocas = False
